In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Dropdown and Label
dropdown_image     = widgets.Dropdown(description =  'Select Image:' , options = ['Select image'] + list(DeconvDF.index))
label_text         = widgets.Text(value='', description='Label:', disabled=True)
dropdown_and_label = widgets.HBox([dropdown_image, label_text])

# Texture Sliders
# slider_gamma   = widgets.IntSlider(min=1,max=1000,step=100,description='Gamma:',value=500)
# slider_thr     = widgets.IntSlider(min=0,max=20,step=1,description='Threshold:',value=0)
# slider_linelen = widgets.FloatSlider(min=0,max=10,step=0.5,description='LineLen/Scale:',value=2.5)
# slider_linegap = widgets.IntSlider(min=0,max=10,step=1,description='LineGap:',value=1)
# sliders        = widgets.HBox([slider_gamma, slider_thr, slider_linelen, slider_linegap])

# # To Add
radiobutton_to_add = widgets.RadioButtons(options=['Yes','No'], description='Add cell to ROIs DataFrame?',value='No')

# Calculate Button
button = widgets.Button(value=False, description='Calculate', disabled=False, button_style='')

# Outputs
output_ROI      = widgets.Output()
output_RES      = widgets.Output()

# Event Handler Functions
def ROI_eventhandler(change):
    image = DeconvDF['Image'][dropdown_image.value]
    
    # Select ROI QT
    %matplotlib qt
    fig,ax = plt.subplots(figsize=(15,15))
    ax.imshow(image,cmap='gray')
    ax.axis('off')
    
    # Plot nuclei contours and centroids
    plot_nuclei_contours(CentroidsDF=Centroids,imgIndex=dropdown_image.value,ax=ax)

    # Skeleton
    fig_s,ax_s = plt.subplots(figsize=(15,15))
    ax_s.imshow(TextureDF['Skeleton'][dropdown_image.value],cmap='gray')
    ax_s.axis('off')
    
    # Plot nuclei contours and centroids
    plot_nuclei_contours(CentroidsDF=Centroids,imgIndex=dropdown_image.value,ax=ax_s)

    # Draw ROI and get Coordinates
    global ROI,mask
    ROI = RoiPoly(color='r')
    ROI.display_roi()
    mask = ROI.get_mask(image) #roi_coordinates = ROI.get_roi_coordinates()
    %matplotlib inline
    
    with output_ROI:
        plt.figure(figsize=(15,10))
        plt.subplot(1,3,1)
        
        # Plot Original
        copia = copy.deepcopy(OriginalDF['Image'][dropdown_image.value])
        auxiliar = mask * OriginalDF['Image'][dropdown_image.value][:,:,2]
        xxx,yyy = np.where(auxiliar != 0)
        helpp = np.zeros_like(auxiliar[min(xxx):max(xxx),min(yyy):max(yyy)])
        auxiliar = np.stack((auxiliar[min(xxx):max(xxx),min(yyy):max(yyy)],helpp,helpp),axis=2)
#         copia[:,:,0] = auxiliar[min(xxx):max(xxx),min(yyy):max(yyy)]
#         copia[:,:,1] = np.zeros_like(copia[:,:,0])
#         copia[:,:,2] = np.zeros_like(copia[:,:,0])
        plt.imshow(auxiliar)
        plt.title('Original')
        plt.axis('off')
        
        # Plot Deconvoluted
        plt.subplot(1,3,2)
        auxiliar = mask * image
        xxx,yyy = np.where(auxiliar != 0)
        plt.imshow(auxiliar[min(xxx):max(xxx),min(yyy):max(yyy)],cmap='gray')
        plt.title('Deconvoluted')
        plt.axis('off')
        
        # Plot Skeleton
        plt.subplot(1,3,3)
        sk = TextureDF['Skeleton'][dropdown_image.value]
        plt.imshow(np.max(sk[min(xxx):max(xxx),min(yyy):max(yyy)])-sk[min(xxx):max(xxx),min(yyy):max(yyy)],cmap='gray')
        plt.title('Skeleton')
        plt.axis('off')
        plt.show()
                
    # Define Texture and Analyse Parameters
    #display(sliders)
    display(radiobutton_to_add)
    #text_ske_eventhandler(change)
    
         
def text_ske_eventhandler(change):
    if type(dropdown_image.value) != int:
        algorithm = 'synthetic'
    else:
        algorithm = 1
    #texture,skeleton,TextureDF = img_getTexture([mask * DeconvDF['Image'][dropdown_image.value],dropdown_image.value,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=change.new, algorithm=1,TextureDF=TextureDF,to_add=radiobutton_to_add.value)
    skeleton = img_preprocessing(image=[OriginalDF['Image'][dropdown_image.value],dropdown_image.value], algorithm='original',parameters=[0.03,500],plot=False)

    with output_ROI:
        print('o')
        plt.clf()
        plt.figure(figsize=(10,10))
        
        x_x,y_y = np.where(texture != 0)
        #plt.imshow(texture[min(x_x):max(x_x),min(y_y):max(y_y)],cmap='gray')
        plt.axis('off')
        plt.imshow(DeconvDF['Image'][dropdown_image.value],cmap='gray')
        plt.show()
#         plt.subplot(1,2,2)
#         x_x,y_y = np.where(skeleton != 0)
#         plt.imshow(skeleton[min(x_x):max(x_x),min(y_y):max(y_y)],cmap='gray')
#         plt.axis('off')
#         plt.show()
            
def RES_eventhandler(change):
    output_RES.clear_output()
    #image = DeconvDF['Image'][dropdown_image.value]
    
    
    with output_RES:
        %matplotlib inline
        
#         if type(dropdown_image.value) != int:
#             texture,skeleton,TextureDF = img_getTexture([mask * DeconvDF['Image'][dropdown_image.value],dropdown_image.value,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_gamma.value, algorithm='synthetic',TextureDF=TextureDF,to_add=radiobutton_to_add.value)  
#         else:
         
        global ImageLinesDF
        %matplotlib qt
        if radiobutton_to_add.value == 'texture':
            ImageLinesDF = analyze_cell([texture, dropdown_image.value],mask,[slider_thr.value,slider_linelen.value,slider_linegap.value],Centroids[dropdown_image.value],DeconvDF,NucleiDeconvDF,True)
        if radiobutton_to_add.value == 'skeleton':
            ImageLinesDF = analyze_cell([skeleton, dropdown_image.value],mask,[slider_thr.value,slider_linelen.value,slider_linegap.value],Centroids[dropdown_image.value],DeconvDF,NucleiDeconvDF,True)
        
        #ResultsDF = analyze_cell([skeleton, dropdown_image.value],mask,[2,2.5,1],Centroids[dropdown_image.value],DeconvDF,NucleiDeconvDF,True)
        ResultsDF = analyze_cell([skeleton, dropdown_image.value,texture],row['ROImask'],[2,2.5,1],Centroids[row['Index']],OriginalDF,DeconvDF,NucleiDeconvDF,True)
        
        quantitative_analysis(ResultsDF)

def dropdown_image_eventhandler(change):
    output_ROI.clear_output()
    output_RES.clear_output()
    
    global img_id
    img_id = dropdown_image.value
    label_text.value = DeconvDF['Label'][dropdown_image.value]
    
    with output_ROI:
        ROI_eventhandler(change)
        #text_ske_eventhandler(change)
        display(button)
        
# OBSERVE    
#radiobuttons_original.observe(radiobuttons_original_eventhandler, names='value')
dropdown_image.observe(dropdown_image_eventhandler, names='value')
                             
# Display initial widgets
display(dropdown_and_label)

# CALCULATE AND ADD ROI
with output_RES:
    button.on_click(RES_eventhandler)
    if radiobutton_to_add.value == 'Yes':
        new  = pd.DataFrame(data = {'Name': [DeconvDF['Name'][img_id]],'Index': [img_id], 'Label': [DeconvDF['Label'][img_id]], 'ROImask': [mask]})
        ROIs = ROIs.append(new,ignore_index=True)

# TABS
tab = widgets.Tab([output_ROI, output_RES])
tab.set_title(0, 'Select ROI'); tab.set_title(1, 'Results')

display(tab)

In [ ]:
# Widget Types
dropdown_image = widgets.Dropdown(description =  'Select Image:' , options = DeconvDF.index)
label_text = widgets.Text(value='', description='Label:', disabled=True)
dropdown_and_label = widgets.HBox([dropdown_image, label_text])
slider_texture = widgets.IntSlider(min=1,max=1000,step=100,description='Gamma:',value=500)
texture, skeleton, filt_contours, contour_out, contour_int = img_getTexture([(255*(DeconvDF['Image'][dropdown_image.value]/np.max(DeconvDF['Image'][dropdown_image.value]))).astype(np.uint8),index,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_texture.value, algorithm='contour',TextureDF=TextureDF,to_add='null')
dropdown_contours = widgets.Dropdown(description =  'Select Contour:', options = list(range(len(filt_contours))))


# Outputs
output_deconv        = widgets.Output()
output_texture       = widgets.Output()
output_cont_int      = widgets.Output()
output_nuclei        = widgets.Output()
output_analyze_cont  = widgets.Output()

# Event Handler Functions     
def slider_texture_eventhandler(change):
    # Get Texture
    #texture,skeleton,TextureDF = img_getTexture(image=[DeconvDF['Image'][dropdown_image.value], dropdown_image.value, DeconvDF['Name'][dropdown_image.value], DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_texture.value, algorithm=1,TextureDF=0,to_add='null')
    texture, skeleton, filt_contours, contour_out, contour_int = img_getTexture([(255*(DeconvDF['Image'][dropdown_image.value]/np.max(DeconvDF['Image'][dropdown_image.value]))).astype(np.uint8),index,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_texture.value, algorithm='contour',TextureDF=TextureDF,to_add='null')
 
    with output_texture:
        output_texture.clear_output()
        
        plt.figure(figsize=(15,15))
        plt.imshow(texture,cmap='gray')
        plt.axis('off')
        plt.show()
        
        plt.figure(figsize=(15,15))
        plt.imshow(skeleton,cmap='gray')
        plt.axis('off')
        plt.show()
        
        display(slider_texture)
        

def test(change):
    a = 2
    
def analyze_contour_eventhandler(change):
    texture, skeleton, filt_contours, contour_out, contour_int = img_getTexture([(255*(DeconvDF['Image'][dropdown_image.value]/np.max(DeconvDF['Image'][dropdown_image.value]))).astype(np.uint8),index,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_texture.value, algorithm='contour',TextureDF=TextureDF,to_add='null')
    with output_analyze_cont:
        output_analyze_cont.clear_output()
        display(dropdown_contours)
        analyze_contour(filt_contours[dropdown_contours.value],[texture,dropdown_image.value],[0,2,1],Centroids[dropdown_image.value]['Centroid'],True)
                                   
def dropdown_image_eventhandler(change):
    output_deconv.clear_output()
    output_texture.clear_output()
    output_nuclei.clear_output()
    output_cont_int.clear_output()
    label_text.value = DeconvDF['Label'][dropdown_image.value]
        
    with output_deconv:
        plt.figure(figsize=(15,15))
        plt.imshow(DeconvDF['Image'][change.new],cmap='gray')
        plt.axis('off')
        plt.show()
    with output_texture:
        slider_texture_eventhandler(change)
    with output_cont_int:
        texture, skeleton, filt_contours, contour_out, contour_int = img_getTexture([(255*(DeconvDF['Image'][dropdown_image.value]/np.max(DeconvDF['Image'][dropdown_image.value]))).astype(np.uint8),index,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_texture.value, algorithm='contour',TextureDF=TextureDF,to_add='null')
        plt.figure(figsize=(15,15))
        plt.imshow(contour_int,cmap='gray')
        plt.axis('off')
        plt.show()
    with output_nuclei:
        plt.figure(figsize=(15,15))
        plt.imshow(NucleiDeconvDF['Image'][change.new],cmap='gray')
        plt.axis('off')
        for index_c,row_c in Centroids[change.new].iterrows():
            plt.plot(row_c['Centroid'][1],row_c['Centroid'][0],'o',color='red',markersize=7)
        plt.show()
    with output_analyze_cont:
        texture, skeleton, filt_contours, contour_out, contour_int = img_getTexture([(255*(DeconvDF['Image'][dropdown_image.value]/np.max(DeconvDF['Image'][dropdown_image.value]))).astype(np.uint8),index,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_texture.value, algorithm='contour',TextureDF=TextureDF,to_add='null')
        #dropdown_contours = widgets.Dropdown(description =  'Select Contour:' , options = list(range(len(filt_contours))))
        display(dropdown_contours)
        analyze_contour_eventhandler(change)
        
        


# OBSERVE    
dropdown_image.observe(dropdown_image_eventhandler, names='value')
slider_texture.observe(slider_texture_eventhandler, names='value')
dropdown_contours.observe(analyze_contour_eventhandler, names='value')
                               
# Display initial widgets
display(dropdown_and_label)


# Tabs
tab = widgets.Tab([output_deconv, output_texture, output_cont_int, output_nuclei, output_analyze_cont])
tab.set_title(0, 'Deconvoluted')
tab.set_title(1, 'Texture')
tab.set_title(2, 'Contours')
tab.set_title(3, 'Nuclei')
tab.set_title(4, 'Analysis')
display(tab)

In [ ]:
# Widget Types
dropdown_image = widgets.Dropdown(description =  'Select Image:' , options = DeconvDF.index)
label_text = widgets.Text(value='', description='Label:', disabled=True)
dropdown_and_label = widgets.HBox([dropdown_image, label_text])
radiobuttons_original = widgets.RadioButtons(options=['all','red', 'blue', 'green'], description='Select channel:')
slider_texture = widgets.IntSlider(min=1,max=1000,step=100,description='Gamma:',value=500)
slider_nbhd_radius = widgets.IntSlider(min=5,max=100,step=5,description='Radius:',value=50)
slider_nbhd_linelen = widgets.IntSlider(min=2,max=15,step=1,description='Line Length:',value=3)
sliders_nbhd = widgets.HBox([slider_nbhd_radius, slider_nbhd_linelen])
button_neigh = widgets.Button(value=False, description='Calculate', disabled=False, button_style='')
dropdown_centroids = widgets.Dropdown(description =  'Select Centroid:' , options = [str(round(x[0],2)) + "," + str(round(x[1],2)) for x in Centroids[img_id]['Centroid']])

# Outputs
output_original      = widgets.Output()
output_deconv        = widgets.Output()
output_texture       = widgets.Output()
output_nuclei        = widgets.Output()
output_neighbourhood = widgets.Output()
output_plots         = widgets.Output()


# Event Handler Functions
def radiobuttons_original_eventhandler(change):
    output_original.clear_output()
    image = OriginalDF['Image'][dropdown_image.value]
    copia = copy.deepcopy(OriginalDF['Image'][dropdown_image.value])
    if change.new == 'all' or radiobuttons_original.value == 'all':
        copia[:,:,0] = image[:,:,2]
        copia[:,:,2] = image[:,:,0]
    if change.new == 'red':
        copia[:,:,0] = image[:,:,2]
        copia[:,:,1] = 0
        copia[:,:,2] = 0
    if change.new == 'green':
        copia[:,:,0] = 0
        copia[:,:,1] = image[:,:,1]
        copia[:,:,2] = 0
    if change.new == 'blue':
        copia[:,:,0] = 0
        copia[:,:,1] = 0
        copia[:,:,2] = image[:,:,0]
    
    with output_original:
        plt.figure(figsize=(15,15))
        plt.imshow(copia,cmap='gray')
        plt.axis('off')
        plt.show()
        display(radiobuttons_original)
        
def slider_texture_eventhandler(change):
    # Get Texture
    texture,skeleton,TextureDF = img_getTexture(image=[DeconvDF['Image'][dropdown_image.value], dropdown_image.value, DeconvDF['Name'][dropdown_image.value], DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_texture.value, algorithm=1,TextureDF=0,to_add='null')

    with output_texture:
        output_texture.clear_output()
        
        plt.figure(figsize=(15,15))
        plt.imshow(texture,cmap='gray')
        plt.axis('off')
        plt.show()
        
        plt.figure(figsize=(15,15))
        plt.imshow(skeleton,cmap='gray')
        plt.axis('off')
        plt.show()
        
        display(slider_texture)

def test(change):
    a = 2
                                    
def slider_nbhd_eventhandler(change):
    output_neighbourhood.clear_output()
    img = TextureDF['Image'][dropdown_image.value]
    
    with output_neighbourhood:
        plt.figure(figsize=(15,15))
        plt.imshow(img,cmap='gray',alpha=0.2)
        plt.axis('off')
        
        WholeImageLinesDF = pd.DataFrame(columns = ['Name'] + ['Label'] + ['Angles'] + ['Distances to Centroid'] + ['Triangle Areas'] + ['Line Lengths'])

        for index_c,row_c in Centroids[dropdown_image.value].iterrows():
            
            # Get features around centroid
            WholeImageLinesDF = FeaturesFromCentroid(image=[img,index],index_c=index_c,centroid=row_c['Centroid'],radius=slider_nbhd_radius.value,threshold=0,line_length=slider_nbhd_linelen.value,line_gap=1,ImageLinesDF=WholeImageLinesDF,plot=True)

            if index_c%5 == 0:
                print("Centroid Index: " + str(index_c) + " Complete: " + str((index_c*100)/len(Centroids[dropdown_image.value])) + "%")


        plt.show()
        #display(slider_nbhd_radius)
        #display(slider_nbhd_linelen)
        display(sliders_nbhd)
        display(button_neigh)
    #return ImageLinesDF
    
def dropdown_plots_eventhandler(change):
    img = TextureDF['Image'][dropdown_image.value]
    output_plots.clear_output()
    with output_plots:
        display(dropdown_centroids)
        plt.figure(figsize=(5,5))
        plt.imshow(img,cmap='gray',alpha=0.2)
        plt.axis('off')
        
        Centroid = tuple(dropdown_centroids.value)
        Centroid = dropdown_centroids.value.split(",")
        Centroid = (float(Centroid[0]),float(Centroid[1]))
        
        ImageLinesDF = pd.DataFrame(columns = ['Name'] + ['Label'] + ['Angles'] + ['Distances to Centroid'] + ['Triangle Areas'] + ['Line Lengths'])

        # Get features around centroid
        #ImageLinesDF = FeaturesFromCentroid(image=img,index=index,index_c=0,centroid=Centroid,radius=slider_nbhd_radius.value,threshold=0,line_length=slider_nbhd_linelen.value,line_gap=1,ImageLinesDF=ImageLinesDF,plot=True)
        ImageLinesDF = FeaturesFromCentroid(image=[img,index],index_c=0,centroid=Centroid,radius=slider_nbhd_radius.value,threshold=0,line_length=slider_nbhd_linelen.value,line_gap=1,ImageLinesDF=ImageLinesDF,plot=True)
        
        # GET HISTOGRAMS
        feats  = list(ImageLinesDF.columns[2:])
        colors = ['#3498DB','#E74C3C','#95A5A6','#ABE6FF','#CA6F1E','#2ECC71']

        s=1
        plt.figure(figsize=(6,4*len(feats)))
        for feat in feats: 

            histog = np.histogram(np.array(ImageLinesDF[feat][0]),bins=30)

            plt.subplot(len(feats),1,s)
            #plt.plot(histog[1][:-1],histog[0],color=colors[labels.index(label)],linewidth=4,label = label,alpha=0.7)
            #plt.subplot(len(feats)+1,2,s+1)
            #plt.plot(histog[1][:-1],histog[0]/(histog[0].sum()),color=colors[labels.index(label)],linewidth=4,label = label,alpha=0.7)
            #plt.plot(histog[1][:-1],(histog[0]-histog[0].min())/(histog[0].max()-histog[0].min()),color=colors[labels.index(label)],linewidth=4,label = label,alpha=0.7)
            if feat == 'Distances to Centroid':
                aux2 = []
                for i in range(len(histog[1])-1):
                    aux2 += [np.pi*(histog[1][i+1])**2 - np.pi*(histog[1][i])**2]
                plt.plot(histog[1][:-1],(histog[0]/np.trapz(histog[0],x=histog[1][:-1]))/aux2,linewidth=4,alpha=0.7)
                #plt.plot(histog[1][:-1],histog[0]/np.trapz(histog[0],x=histog[1][:-1]),color=colors[labels.index(label)],linewidth=4,label = label,alpha=0.7)

            else:
                plt.plot(histog[1][:-1],histog[0]/np.trapz(histog[0],x=histog[1][:-1]),linewidth=4,alpha=0.7)

            #if feat == 'Triangle Areas':
                #plt.xlim([-5, 80])

            #if feat == 'Line Lengths':
                #plt.xlim([1.5, 10])
            
    
            #plt.subplot(len(feats),1,s)
            plt.grid(alpha=0.3)
            plt.ylabel('Probability',fontsize=16)
            if feats.index(feat) == 0:
                plt.xlabel('Angles (º)')
            if feats.index(feat) == 1:
                plt.xlabel('Pixels')
            if feats.index(feat) == 2:
                plt.xlabel('Pixels')
            plt.title('Histogram - ' + feat,fontsize=20)
            plt.tight_layout()

            #plt.subplot(len(feats)+1,2,s+1)
            #plt.legend()
            #plt.grid(alpha=0.3)
            plt.title('Probability Density Function - ' + feat,fontsize=20)
            #plt.tight_layout()

            s += 1
            
        plt.show()

def dropdown_image_eventhandler(change):
    output_original.clear_output()
    output_deconv.clear_output()
    output_texture.clear_output()
    output_nuclei.clear_output()
    output_neighbourhood.clear_output()
    output_plots.clear_output()
    #img_id = dropdown_image.value
    label_text.value = DeconvDF['Label'][dropdown_image.value]
    
    with output_original:
        #display(radiobuttons_original)
        radiobuttons_original_eventhandler(change)
    with output_deconv:
        plt.figure(figsize=(15,15))
        plt.imshow(DeconvDF['Image'][change.new],cmap='gray')
        plt.axis('off')
        plt.show()
    with output_texture:
        #display(slider_texture)
        slider_texture_eventhandler(change)
    with output_nuclei:
        plt.figure(figsize=(15,15))
        plt.imshow(NucleiDeconvDF['Image'][change.new],cmap='gray')
        plt.axis('off')
        for index_c,row_c in Centroids[change.new].iterrows():
            plt.plot(row_c['Centroid'][1],row_c['Centroid'][0],'o',color='red',markersize=7)
        plt.show()
    with output_neighbourhood:
        #display(slider_nbhd_radius)
        #display(slider_nbhd_linelen)
        display(sliders_nbhd)
        display(button_neigh)
        #slider_nnbhd_eventhandler(change)
        #button_neigh.on_click(slider_nbhd_eventhandler)
    with output_plots:
        display(dropdown_centroids)
        #dropdown_plots_eventhandler(change)
        

    
# OBSERVE    
radiobuttons_original.observe(radiobuttons_original_eventhandler, names='value')
dropdown_image.observe(dropdown_image_eventhandler, names='value')
slider_texture.observe(slider_texture_eventhandler, names='value')
slider_nbhd_radius.observe(test, names='value')
slider_nbhd_linelen.observe(test, names='value')
button_neigh.observe(slider_nbhd_eventhandler, names='value')
dropdown_centroids.observe(dropdown_plots_eventhandler, names='value')
                               
# Display initial widgets
display(dropdown_and_label)

with output_neighbourhood:
    button_neigh.on_click(slider_nbhd_eventhandler)

# Tabs
tab = widgets.Tab([output_original, output_deconv, output_texture, output_nuclei, output_neighbourhood, output_plots])
tab.set_title(0, 'Original')
tab.set_title(1, 'Deconvoluted')
tab.set_title(2, 'Texture')
tab.set_title(3, 'Nuclei')
tab.set_title(4, 'Neighbourhood')
tab.set_title(5, 'Plots')
display(tab)